# Hamiltonian Monte Carlo or Hybrid Monte Carlo (HMC)

The GPy package, along with other packages like PyMC, use Hamiltonian Monte Carlo for sampling from a posterior. HMC is modified version of the Metropolis Hastings (MH) where each step in the iteration is chosen by the simulated energy of a Hamiltonian dynamical system. The benefit to HMC is that its proposals have high acceptance rates, regardless of the dimensionality of the posterior (number of hyperparameters).
<br><br>
Before we begin looking into HMC, we review the principles of calculus of variations, lagrangians, the principle of stationary action, and Hamiltonian Mechanics as a build up to HMC.

---
## Calculus of Variations
Q: What is the shortest path between two points on a plane? If we zoom into any curve we can approximate the length of a straight line segment with

$$
ds = \sqrt{(dx)^2+(dy)^2} = \sqrt{1+(dy)^2}
$$

Then the total length of the path traveled would be

$$
\int_{x_1}^{x_2}\sqrt{1+(y')^2}
$$


The integrand is the **functional** with a generalization $\int_{x_1}^{x_2} F(x,y,y^2)dx$. $F$ outputs distance.


---
## Euler-Lagrange equation

Recall: the extremum is $f'(x_0)$, $df = f'(x_0)dx = 0$.

Say we have two end points, $(x_1,y_1)$ and $(x_2,y_2)$. Say we have a function $y(x)$ that goes through the two endpoints and we add a small offset: $\bar{y}(x) = y(x) + \epsilon\delta(x)$ where $\epsilon$ is the variational parameter. For interpolation, we set $\delta(x_1) = \delta(x_2) = 0$ constant so we have fixed endpoints. Let's do some calculus:
\begin{align}
\frac{d\bar{y}}{d\epsilon}=\delta(x), \frac{\bar{y}'}{d\epsilon} = \frac{d}{d\epsilon}(y'(x)+\epsilon\delta'(x)) = \delta'(x)
\end{align}

Back to the functional:
$$
I(y) = \int_{x_1}^{x_2} F(x,y,y') dx
$$

$$
\frac{dI}{d\epsilon}_{\epsilon = 0} = \int_{x_1}^{x_2}\left[\frac{dF}{d\bar{y}}\frac{d\bar{y}}{d\epsilon} + \frac{dF}{d\bar{y'}}\frac{d\bar{y'}}{d\epsilon} \right]dx = \int_{x_1}^{x_2} \left( \frac{dF}{dy}\delta(x) + \frac{dF}{d\bar{y'}}\delta'(x) \right)dx =\int_{x_1}^{x_2}\frac{dF}{d\bar{y}}\delta(x)dx + \int_{x_1}^{x_2}\frac{dF}{d\bar{y'}}\delta'(x)dx
$$

<br><br>

Let $v'=\delta'(x)$, $u=\frac{dF}{d\bar{y}'}$, $v = \delta(x)$, $u' = \frac{d}{dx}\left(\frac{dF}{d\bar{y}'}\right)$. Also, we set $\epsilon = 0$, so $\bar{y} = y$. Then,

$$
0 = \frac{dI}{d\epsilon}\vert_{\epsilon = 0}
$$

$$
0 = \int_{x_1}^{x_2}\frac{dF}{dy}\delta(x)dx + \frac{dF}{dy'}\delta(x)\vert_{x_1}^{x_2} - \int_{x_1}^{x_2}\frac{d}{dx}\left(\frac{dF}{dy'}\right)\delta(x_1)dx
$$

$$
0 = \int_{x_1}^{x_2}\frac{dF}{dy}\delta(x)dx - \int_{x_1}^{x_2}\frac{d}{dx}\left(\frac{dF}{dy'}\right)\delta(x)dx = \int_{x_1}^{x_2}\delta(x)\left[\frac{dF}{dy} - \frac{d}{dx}\left(\frac{dF}{dy'}\right) \right]dx
$$

We want the last line to be a minimum for any $\delta(x)$. Thus, the last line only holds true when

$$
\frac{dF}{dy} - \frac{d}{dx}\left(\frac{dF}{dy'}\right) = 0
$$

This is the Euler-Lagrange equation. Solutions to this equation are stationary points (min, max) to the action functional, a function which takes trajectory (path) of a system as its argument and has a real number result.

---
## Lagrangian Equation
The Lagrangian is $L=T-V$, which is the Kinetic-Potential, the total energy of a system and the Euler-Lagrance quation is

$$
\frac{dL}{dx} = \frac{d}{dt}\left(\frac{dL}{d\dot x}\right)
$$

$L = T-V$, $t$ is time, $x$ is position, $\dot x = x' = \frac{dx}{dt}$, $\ddot x = x''$.

**Ex 1:**

In a frictionless spring-mass system, $L = \frac{1}{2}m\dot x^2 - \frac{1}{2}kx^2$. Then,

$$
\frac{dL}{dx} = \frac{d}{dx}\left(\frac{1}{2}m\dot x^2 - \frac{1}{2}kx^2\right) = -kx
$$

$$
\frac{d}{dt}\left(\frac{dL}{d\dot x}\right) = m\ddot x
$$

so $m\ddot x = -kx$.

**Ex2:**

Say we have a 3D particle moving with potential energy $V(x_1,x_2,x_3)$. Then

$$
L = \frac{1}{2}m(\dot x_1^2 + \dot x_2^2 + \dot x_3^2) - V(x_1,x_2,x_3)
$$

So

$$
\nabla_t\nabla_{\dot x} L = m[\ddot x_1, \ddot x_2, \ddot x_3]^T = m\ddot x
$$

$$
\nabla_x L = -k [x_1, x_2, x_3]^T = -kx
$$

<br>

**Deriving Lagrangian's equation:**

Say we have $F = ma$. Then $-\frac{d}{dx}V = mx$ and so $\frac{-dV}{dx} - \frac{d}{dt}m\dot x = 0$. Then,

$$
-\frac{dV}{dx} - \frac{d}{dt}\left[\frac{d}{d\dot x}\frac{m\dot x^2}{2}\right] = 0
$$

$$
\frac{d}{dx}\left[\frac{m\dot x^2}{2} - V\right] - \frac{d}{dt}\left[\frac{d}{d\dot x} \left(\frac{m\dot x^2}{2} - V\right)\right] = 0
$$

We have $T = \frac{m\dot x^2}{2}$ so

$$
\frac{d}{dx}\left[T-V\right] - \frac{d}{dt}\left[\frac{d}{d\dot x}(T-V)\right] = 0
$$

---
## Principle of Stationary Action
A functional represents all paths between endpoints of the integral and a stationary function is the shortest path. A solution to any such functional is the Euler-Lagrange equation. Let $L$ be the Lagrangian. Then the functional of $L$ with respect to time is 

$$
S = \int_{t_1}^{t_2} L(t,x,\dot x)dt
$$

which is called the action as the dimensions are Energy*Time. Since the stationary function is given by the Euler-Lagrange equations, what this means is that objects take the path of the least action or least energy across time. This is the principle of stationary action (Hamilton's principle).

<br>

Regardless of the coordinate system(s) we use, each generalized coordinate has an associated generalized velocity $\dot q_k = dq_k/dt$. The Lagrangian is

$$
L = L(t,q_k,\dot q_k)
$$

while the action over the Lagrangian is

$$
S[q_k(t)] = \int_{t_1}^{t_2} L(t,q_k,\dot q_k) dt
$$

and we rewrite a generalized Euler-Lagrange as

$$
\frac{dL}{d\dot q_k} - \frac{d}{dt}\frac{dL}{d\dot q_k} = 0, k = 1,2,\ldots,K
$$

---
## Hamiltonian Mechanics
- Lagrangian: $L = T-V$
- Hamiltonian: $H = \dot q_k p_k - L = T + V$ where $p_k = \frac{dL}{d\dot q_k}$ is the generalized momentum.

With $L(t, q_k, \dot q_k)$, 

$$
\frac{dL}{dt} = \frac{dL}{dq_k}\frac{dq_k}{dt} + \frac{dL}{d\dot q_k}\frac{d\dot q_k}{dt} + \frac{dL}{dt} = \frac{d}{dt}\frac{dL}{d\dot q_k}\dot q_k + \frac{dL}{d\dot q_k}\ddot q_k  + \frac{dL}{dt} = \left(\frac{d}{dt}\frac{dL}{dq_k}\right)\dot q + \left(\frac{d}{dt}\dot q_k\right)\frac{dL}{d\dot q_k} + \frac{dL}{dt} = \frac{d}{dt}\frac{dL}{d\dot q_k}\dot q_k + \frac{dL}{d\dot q_k} \ddot q_k + \frac{dL}{dt}
$$

$$
= \frac{d}{dt}\left(\dot q_k\frac{dL}{d\dot q_k}\right) + \frac{dL}{dt}
$$

$$
0 = \frac{d}{dt}\left(\dot q_k\frac{dL}{d\dot q_k} - L\right) + \frac{dL}{dt}
$$

where $\frac{dL}{d\dot q_k} = p_k = m\dot q_k$. We then let

$$
H = \dot q_k p_k - L = \dot q_k p_k - \left(\frac{1}{2}m\dot q_k^2 + V(q_k) \right) = \dot q_k^2m - \frac{1}{2}\dot q_k^2 m + V(q_k) = \frac{1}{2}m\dot q_k^2 + V(q_k) = T+V
$$

<br>

Let $W$ denote work, we can write the potential energy as

$$
V = -W = -Fq_k = -\frac{dp_k}{dt}q_k = -\dot p_k q_k
$$

We can write the kinetic energy as (with $p_k = m\dot q_k$)

$$
T = \frac{1}{2}m\dot q_k^2 = \frac{p_k^2}{2m}
$$

Together,

$$
H = T+V = \frac{p_k^2}{2m} - \dot p_k q_k
$$

and therefore, the Hamilton's equations,

$$
\frac{dH}{dq_k} = -\dot p_k = -\frac{dp_k}{dt}
$$

$$
\frac{dH}{dp_k} = \frac{p_k}{m} = \dot q_k = \frac{dq_k}{dt}
$$

<br><br>

**Sources:**
- https://gregorygundersen.com/blog/2020/05/10/euler-lagrange/
- https://gregorygundersen.com/blog/2020/06/14/lagrangian-hamiltonian/

https://www.tweag.io/blog/2020-08-06-mcmc-intro3/